# Node

> TODO fill in description

In [1]:
#|default_exp node

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type, Callable, Dict, Any
from enum import Enum

from fbdev import BaseComponent, utils, BasePort, BaseInputPort, BaseOutputPort, Edge
from fbdev.utils import Expando, StateHandler, StateView

In [5]:
import time
import fbdev
from fbdev import Packet

In [6]:
#|export
__all__ = ['BaseNode', 'Node']

In [7]:
#|export
class BaseNode:
    pass

In [8]:
#|hide
show_doc(fbdev.node.Node)

---

### Node

>      Node (component_type:Type[ForwardRef('BaseComponent')])

*Initialize self.  See help(type(self)) for accurate signature.*

In [9]:
#|export

PortState = Enum('PortState', ['OPEN', 'CLOSED'])

class Node:
    def __init__(self, component_type: Type['BaseComponent']):
        self.component_type = component_type
        self.component_process = None
        self._id = None  # TODO: implement Node._id
                
        self.__initialise()

    def __initialise(self):
        """Instantiate the component process"""
        self.component_process = self.component_type(self)
        
        self.config_input_edges = Expando(obj_name="config_input_edges")
        self.input_edges = Expando(obj_name="input_edges")
        self.output_edges = Expando(obj_name="output_edges")
        
        self.port_states = Expando({
            'config_input': {},
            'input': {},
            'output': {}
        }, obj_name="port_states")
        
        self._edge_disconnect_events = Expando({
            'config_input': {},
            'input': {},
            'output': {}
        }, obj_name="edge_disconnected_events")
        
        self.running = StateHandler(False)
        self.executing = StateHandler(False)
        self._stop_running_event = asyncio.Event()
        
        for port in self.component_process.config_inputs.values():
            self.config_input_edges._attrs[port.name] = None
            self.port_states['config_input'][port.name] = StateHandler(PortState.OPEN, PortState)
            self._edge_disconnect_events['config_input'][port.name] = asyncio.Event()
        for port in self.component_process.inputs.values():
            self.input_edges._attrs[port.name] = None
            self.port_states['input'][port.name] = StateHandler(PortState.OPEN, PortState)
            self._edge_disconnect_events['input'][port.name] = StateHandler(True)
        for port in self.component_process.outputs.values():
            self.output_edges._attrs[port.name] = None
            self.port_states['output'][port.name] = StateHandler(PortState.OPEN, PortState)
            self._edge_disconnect_events['output'][port.name] = asyncio.Event()
                
    def reset(self):
        self.__initialise()
                
    def __connect_edge(self, connection_type: str, is_input: bool, port: BasePort, edge: 'Edge'):
        edge_dict = getattr(self, f"{connection_type}_edges")
        if port.name in edge_dict: raise ValueError(f"{connection_type} port '{port.name}' is already connected") #TODO proper exception
        
        edge_dict[port.name] = edge
        edge_disconnected_event = self._edge_disconnect_events[connection_type][port.name]
        port_is_open = self.port_states[connection_type][port.name].get_state_event(PortState.OPEN)
        
        if is_input: edge._output_node = self
        else: edge._input_node = self
        asyncio.run(self.__handle_edge_port_connection(is_input, edge_dict, edge, port, edge_disconnected_event, port_is_open))

    def connect_config_input_edge(self, port_name: str, edge: Edge):
        self.__connect_edge('config_input', True, self.component_process.config_inputs[port_name], edge)
    def connect_input_edge(self, port_name: str, edge: Edge):
        self.__connect_edge('input', True, self.component_process.inputs[port_name], edge)
    def connect_output_edge(self, port_name: str, edge: Edge):
        self.__connect_edge('output', False, self.component_process.outputs[port_name], edge)
        
    def __disconnect_edge(self, connection_type: str, is_input: bool, port_name: str):
        edge_dict = getattr(self, f"{connection_type}_edges")
        if port_name in edge_dict: raise ValueError(f"{connection_type} port '{port_name}' is already connected") #TODO proper exception
        
        edge: Edge = edge_dict[port_name]
        if is_input: edge._output_node = None
        else: edge._input_node = None        
        self._edge_disconnect_events[connection_type][port_name].set() # shuts down the channel

    def disconnect_config_input_edge(self, port_name: str):
        self.__disconnect_edge('config_input', True, port_name)
    def disconnect_input_edge(self, port_name: str):
        self.__disconnect_edge('input', True, port_name)
    def disconnect_output_edge(self, port_name: str):
        self.__disconnect_edge('output', False, port_name)
        
    async def __handle_edge_port_connection(self, is_input:bool, edge_dict:dict, edge:Edge, port:BaseInputPort, edge_disconnect:asyncio.Event, port_is_open:asyncio.Event):
        while not edge_disconnect.is_set():
            if is_input: port.load(edge.unload())
            else: edge.load(port.unload())
            port_available = port.full_state.get_state_event(False)
            edge_enabled = edge.enabled.get_state_event(True)
            edge_available = edge.full.get_state_event(False)
            await utils.await_multiple_events(port_is_open, port_available, edge_available, edge_enabled) # Important this is last, so that you don't have one final unload before a stop event
        edge_disconnect.clear()
        edge_dict[port.name] = None

    def open_config_input_ports(self, name):
        self.port_states.config_input[name].set()
    def open_input_ports(self, name):
        self.port_states.input[name].set()
    def open_output_ports(self, name):
        self.port_states.output[name].set()
        
    def close_config_input_ports(self, name):
        self.port_states.config_input[name].clear()
    def close_input_ports(self, name):
        self.port_states.input[name].clear()
    def close_output_ports(self, name):
        self.port_states.output[name].clear()
            
    async def start(self):
        self.running.set(True)
            
        while not self._stop_running_event.is_set():
            await self.component_process.update_config()
            self.executing.set(True)
            await self.component_process.execute()
            self.executing.set(False)
        
        self._stop_running_event.clear()
        self.running.set(False)
             
    def stop(self):
        self._stop_running_event.set()
    

In [22]:
class MyComponent(BaseComponent):
    config_input_specs = [{'name':'msg', 'default':'Hello'}] 
    input_specs = []
    output_specs = [{'name':'out1', 'port_type':fbdev.DequeOutputPort, 'port_kwargs':{'maxsize':5}}]

    def __init__(self, node):
        super().__init__(node)
        self._count = 0
    
    async def execute(self):
        self._count += 1
        await self.outputs.out1.put(f"Hello #{self._count}")

node = Node(MyComponent)
node.initialise()
asyncio.create_task(node.start())
await node.component_process.outputs.out1.full.wait(True)
node.stop()

for i in range(5):
    print(node.component_process.outputs.out1.unload().get_data())

Hello #1
Hello #2
Hello #3
Hello #4
Hello #5


In [13]:
#|hide
show_doc(fbdev.node.ProxyProcessNode)

---

### ProxyProcessNode

>      ProxyProcessNode ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [14]:
#|export
class ProxyProcessNode(Node):
    def __init__(self):
        pass

In [15]:
#|hide
show_doc(fbdev.node.ProcessNode)

---

### ProcessNode

>      ProcessNode ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [16]:
#|export
class ProcessNode(Node):
    def __init__(self):
        pass